In [ ]:
import matplotlib.pyplot as plt
import scanpy as sc
from matplotlib.pyplot import MultipleLocator
import seaborn as sns
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

In [ ]:
import os
import pandas as pd
import scipy.io
import scipy.sparse
import anndata

# Path to the folder containing the files
folder_path = "./GSE199563"

# Initialize a list to store individual AnnData objects
anndata_list = []

# Loop through all the files in the folder
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith("matrix.mtx.gz"):
            # Extract the sample prefix (e.g., GSM597235_scR101)
            sample_prefix = "_".join(file.split("_")[:2])
            
            # Define file paths
            matrix_path = os.path.join(root, f"{sample_prefix}_matrix.mtx.gz")
            barcodes_path = os.path.join(root, f"{sample_prefix}_barcodes.tsv.gz")
            features_path = os.path.join(root, f"{sample_prefix}_features.tsv.gz")
            
            # Debug: Print file paths
            print(f"Processing sample: {sample_prefix}")
            print(f"Matrix file: {matrix_path}")
            print(f"Barcodes file: {barcodes_path}")
            print(f"Features file: {features_path}")
            
            # Check if all necessary files exist
            if not (os.path.exists(matrix_path) and os.path.exists(barcodes_path) and os.path.exists(features_path)):
                print(f"Missing files for sample: {sample_prefix}, skipping...")
                continue

            # Read barcodes
            barcodes = pd.read_csv(barcodes_path, header=None, sep="\t")[0].tolist()
            
            # Read features
            features = pd.read_csv(features_path, header=None, sep="\t")[1].tolist()
            
            # Read matrix
            matrix = scipy.io.mmread(matrix_path)
            matrix = scipy.sparse.csr_matrix(matrix)  # Convert to CSR format
            
            # Debug: Print dimensions
            print(f"Matrix shape: {matrix.shape}")
            print(f"Number of barcodes: {len(barcodes)}")
            print(f"Number of features: {len(features)}")
            
            # Check consistency
            if matrix.shape[0] != len(features):
                print(f"Gene mismatch in {sample_prefix}: Matrix rows ({matrix.shape[0]}) != Features ({len(features)})")
                continue
            if matrix.shape[1] != len(barcodes):
                print(f"Cell mismatch in {sample_prefix}: Matrix cols ({matrix.shape[1]}) != Barcodes ({len(barcodes)})")
                continue
            
            # Ensure unique barcodes and features
            unique_barcodes = [f"{barcode}_{sample_prefix}" for barcode in barcodes]
            unique_features = [f"{feature}_{i}" for i, feature in enumerate(features)]
            matrix=matrix.T
            # Create AnnData object
            adata = anndata.AnnData(X=matrix)
            adata.obs_names = unique_barcodes
            adata.var_names = unique_features
            adata.obs["sample"] = sample_prefix  # Add sample prefix as metadata
            
            # Append to the list
            anndata_list.append(adata)
            print(f"Successfully processed sample: {sample_prefix}")

# Combine all AnnData objects into one
if anndata_list:
    print("Concatenating all AnnData objects...")
    combined_adata = anndata.concat(anndata_list, axis=0)

    # Save as .h5ad file
    output_path = "GSE199563_combined.h5ad"
    combined_adata.write(output_path)
    print(f"Combined AnnData object saved as '{output_path}'.")
else:
    print("No valid AnnData objects to concatenate. Please check your data.")


In [ ]:
#QC
adata=combined_adata.copy()
adata.var['mt'] = adata.var_names.str.startswith('mt-')  
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], inplace=True)
adata.var_names_make_unique()

sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

min_genes = 200   
max_genes = 2500  
max_mito = 5      

adata = adata[adata.obs.n_genes_by_counts > min_genes, :]
adata = adata[adata.obs.n_genes_by_counts < max_genes, :]
adata = adata[adata.obs.pct_counts_mt < max_mito, :]

adata.raw = adata
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [ ]:
genes = ["Tox","Pdcd1","Ctla4","Tcf7"]

# Define a color map for the genes
gene_colors = {
    "Tox": "blue",
    "Tcf7": "green",
    "Pdcd1": "red",
    "Ctla4": "purple"
}

# Extract the mean expression for each gene, condition, and time point
results = []
for gene in genes:
    naive_mean = adata[adata.obs['condition'].str.contains("Naïve")].to_df()[gene].mean()
    for group in adata.obs['condition'].unique():
        if any(cond in group for cond in ["Arm", "Cl"]):
            condition = "Arm" if "Arm" in group else "Cl"
            for time_point in ['Naïve', "D8", "D15", "D30"]:
                if time_point in group:
                    mean_expr = adata[adata.obs['condition'] == group].to_df()[gene].mean()
                    if not any(r['Gene'] == gene and r['Condition'] == condition and r['Time'] == time_point for r in results):
                        results.append({"Gene": gene, "Condition": condition, "Time": time_point, "Expression": mean_expr})
    # Add Naïve mean for both Arm and Cl
    results.append({"Gene": gene, "Condition": "Arm", "Time": "Naïve", "Expression": naive_mean})
    results.append({"Gene": gene, "Condition": "Cl", "Time": "Naïve", "Expression": naive_mean})

# Create a DataFrame for visualization
data = pd.DataFrame(results)

# Ensure time points are ordered correctly
data['Time'] = pd.Categorical(data['Time'], categories=['Naïve', "D8", "D15", "D30"], ordered=True)

# Sort the data by Gene, Condition, and Time to enforce plotting order
data = data.sort_values(by=['Gene', 'Condition', 'Time'])

# Set up the style for publication-quality figures
sns.set_theme(style="white", context="talk")

# Create a figure and axis
plt.figure(figsize=(7, 4))

# Generate the line plot
for gene in genes:
    subset = data[data['Gene'] == gene]
    color = gene_colors[gene]  # Use the specified color for the gene
    plt.plot(
        subset[subset['Condition'] == 'Arm']['Time'], 
        subset[subset['Condition'] == 'Arm']['Expression'], 
        label=f"{gene} - Arm", 
        marker='o', 
        linestyle='-', 
        color=color
    )
    plt.plot(
        subset[subset['Condition'] == 'Cl']['Time'], 
        subset[subset['Condition'] == 'Cl']['Expression'], 
        label=f"{gene} - Cl", 
        marker='s', 
        linestyle='--', 
        color=color
    )
    
# Customize the plot aesthetics
plt.xlabel("Time (Days)")
plt.ylabel("Mean Expression")
plt.title("Gene Expression Across Conditions and Time Points")
plt.legend(title="Gene - Condition", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

# Save the figure as a high-quality image
plt.savefig("Exhuastion_gene_expression.pdf", dpi=300, bbox_inches='tight')

# Show the plot
plt.show()

In [ ]:
genes = ["Uty", "Kdm5d", "Usp9y", "Ddx3y"]

# Define a color map for the genes
gene_colors = {
    "Uty": "blue",
    "Kdm5d": "green",
    "Usp9y": "red",
    "Ddx3y": "purple"
}

# Extract the mean expression for each gene, condition, and time point
results = []
for gene in genes:
    naive_mean = adata[adata.obs['condition'].str.contains("Naïve")].to_df()[gene].mean()
    for group in adata.obs['condition'].unique():
        if any(cond in group for cond in ["Arm", "Cl"]):
            condition = "Arm" if "Arm" in group else "Cl"
            for time_point in ['Naïve', "D8", "D15", "D30"]:
                if time_point in group:
                    mean_expr = adata[adata.obs['condition'] == group].to_df()[gene].mean()
                    if not any(r['Gene'] == gene and r['Condition'] == condition and r['Time'] == time_point for r in results):
                        results.append({"Gene": gene, "Condition": condition, "Time": time_point, "Expression": mean_expr})
    # Add Naïve mean for both Arm and Cl
    results.append({"Gene": gene, "Condition": "Arm", "Time": "Naïve", "Expression": naive_mean})
    results.append({"Gene": gene, "Condition": "Cl", "Time": "Naïve", "Expression": naive_mean})

# Create a DataFrame for visualization
data = pd.DataFrame(results)

# Ensure time points are ordered correctly
data['Time'] = pd.Categorical(data['Time'], categories=['Naïve', "D8", "D15", "D30"], ordered=True)

# Sort the data by Gene, Condition, and Time to enforce plotting order
data = data.sort_values(by=['Gene', 'Condition', 'Time'])

# Set up the style for publication-quality figures
sns.set_theme(style="white", context="talk")

# Create a figure and axis
plt.figure(figsize=(7, 4))

# Generate the line plot
for gene in genes:
    subset = data[data['Gene'] == gene]
    color = gene_colors[gene]  # Use the specified color for the gene
    plt.plot(
        subset[subset['Condition'] == 'Arm']['Time'], 
        subset[subset['Condition'] == 'Arm']['Expression'], 
        label=f"{gene} - Arm", 
        marker='o', 
        linestyle='-', 
        color=color
    )
    plt.plot(
        subset[subset['Condition'] == 'Cl']['Time'], 
        subset[subset['Condition'] == 'Cl']['Expression'], 
        label=f"{gene} - Cl", 
        marker='s', 
        linestyle='--', 
        color=color
    )
    
# Customize the plot aesthetics
plt.xlabel("Time (Days)")
plt.ylabel("Mean Expression")
plt.title("Gene Expression Across Conditions and Time Points")
plt.legend(title="Gene - Condition", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

# Save the figure as a high-quality image
plt.savefig("ChrY_gene_expression.pdf", dpi=300, bbox_inches='tight')

# Show the plot
plt.show()